<a href="https://colab.research.google.com/github/mohamedEmad23/Colab_Projects/blob/main/COCO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COCO data loading

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip -O coco_train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip -O coco_val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O coco_ann2017.zip


--2025-03-18 19:29:36--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.49.188, 3.5.9.11, 3.5.8.15, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.49.188|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘coco_train2017.zip’

coco_train2017.zip   15%[==>                 ]   2.72G  96.2MB/s    eta 2m 45s 

In [ ]:
from zipfile import ZipFile, BadZipFile
import os
def extract_zip_file(extract_path):
    try:
        with ZipFile(extract_path+".zip") as zfile:
            zfile.extractall(extract_path)
        # remove zipfile
        zfileTOremove=f"{extract_path}"+".zip"
        if os.path.isfile(zfileTOremove):
            os.remove(zfileTOremove)
        else:
            print("Error: %s file not found" % zfileTOremove)
    except BadZipFile as e:
        print("Error:", e)
extract_train_path = "./coco_train2017"
extract_val_path = "./coco_val2017"
extract_ann_path="./coco_ann2017"
extract_zip_file(extract_train_path)
extract_zip_file(extract_val_path)
extract_zip_file(extract_ann_path)

In [ ]:
from collections import defaultdict
import json
import numpy as np
class COCOParser:
    def __init__(self, anns_file, imgs_dir):
        with open(anns_file, 'r') as f:
            coco = json.load(f)

        self.annIm_dict = defaultdict(list)
        self.cat_dict = {}
        self.annId_dict = {}
        self.im_dict = {}
        self.licenses_dict = {}
        for ann in coco['annotations']:
            self.annIm_dict[ann['image_id']].append(ann)
            self.annId_dict[ann['id']]=ann
        for img in coco['images']:
            self.im_dict[img['id']] = img
        for cat in coco['categories']:
            self.cat_dict[cat['id']] = cat
        for license in coco['licenses']:
            self.licenses_dict[license['id']] = license
    def get_imgIds(self):
        return list(self.im_dict.keys())
    def get_annIds(self, im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        return [ann['id'] for im_id in im_ids for ann in self.annIm_dict[im_id]]
    def load_anns(self, ann_ids):
        im_ids=ann_ids if isinstance(ann_ids, list) else [ann_ids]
        return [self.annId_dict[ann_id] for ann_id in ann_ids]
    def load_cats(self, class_ids):
        class_ids=class_ids if isinstance(class_ids, list) else [class_ids]
        return [self.cat_dict[class_id] for class_id in class_ids]
    def get_imgLicenses(self,im_ids):
        im_ids=im_ids if isinstance(im_ids, list) else [im_ids]
        lic_ids = [self.im_dict[im_id]["license"] for im_id in im_ids]
        return [self.licenses_dict[lic_id] for lic_id in lic_ids]

In [ ]:
cocoTrain_annotations_file="/content/coco_ann2017/annotations/instances_train2017.json"
cocoTrain_images_dir="/content/coco_train2017/train2017"
cocoTrain= COCOParser(cocoTrain_annotations_file, cocoTrain_images_dir)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
# define a list of colors for drawing bounding boxes
color_list = ["pink", "red", "teal", "blue", "orange", "yellow", "black", "magenta","green","aqua"]*10
num_imgs_to_disp = 4
total_images = len(cocoTrain.get_imgIds()) # total number of images
sel_im_idxs = np.random.permutation(total_images)[:num_imgs_to_disp]
img_ids = cocoTrain.get_imgIds()
selected_img_ids = [img_ids[i] for i in sel_im_idxs]
ann_ids = cocoTrain.get_annIds(selected_img_ids)
im_licenses = cocoTrain.get_imgLicenses(selected_img_ids)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))
ax = ax.ravel()
for i, im in enumerate(selected_img_ids):
    image = Image.open(f"{cocoTrain_images_dir}/{str(im).zfill(12)}.jpg")
    ann_ids = cocoTrain.get_annIds(im)
    annotations = cocoTrain.load_anns(ann_ids)
    for ann in annotations:
        bbox = ann['bbox']
        x, y, w, h = [int(b) for b in bbox]
        class_id = ann["category_id"]
        class_name = cocoTrain.load_cats(class_id)[0]["name"]
        license = cocoTrain.get_imgLicenses(im)[0]["name"]
        color_ = color_list[class_id]
        rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor=color_, facecolor='none')
        t_box=ax[i].text(x, y, class_name,  color='red', fontsize=10)
        t_box.set_bbox(dict(boxstyle='square, pad=0',facecolor='white', alpha=0.6, edgecolor='blue'))
        ax[i].add_patch(rect)

    ax[i].axis('off')
    ax[i].imshow(image)
    ax[i].set_xlabel('Longitude')
    ax[i].set_title(f"License: {license}")
plt.tight_layout()
plt.show()

In [ ]:
cocoVal_annotations_file="/content/coco_ann2017/annotations/instances_val2017.json"
cocoVal_images_dir="/content/coco_val2017/val2017"
cocoVal= COCOParser(cocoVal_annotations_file, cocoVal_images_dir)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
# define a list of colors for drawing bounding boxes
color_list = ["pink", "red", "teal", "blue", "orange", "yellow", "black", "magenta","green","aqua"]*10
num_imgs_to_disp = 4
total_images = len(cocoVal.get_imgIds()) # total number of images
sel_im_idxs = np.random.permutation(total_images)[:num_imgs_to_disp]
img_ids = cocoVal.get_imgIds()
selected_img_ids = [img_ids[i] for i in sel_im_idxs]
ann_ids = cocoVal.get_annIds(selected_img_ids)
im_licenses = cocoVal.get_imgLicenses(selected_img_ids)
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))
ax = ax.ravel()
for i, im in enumerate(selected_img_ids):
    image = Image.open(f"{cocoVal_images_dir}/{str(im).zfill(12)}.jpg")
    ann_ids = cocoVal.get_annIds(im)
    annotations = cocoVal.load_anns(ann_ids)
    for ann in annotations:
        bbox = ann['bbox']
        x, y, w, h = [int(b) for b in bbox]
        class_id = ann["category_id"]
        class_name = cocoVal.load_cats(class_id)[0]["name"]
        license = cocoVal.get_imgLicenses(im)[0]["name"]
        color_ = color_list[class_id]
        rect = plt.Rectangle((x, y), w, h, linewidth=2, edgecolor=color_, facecolor='none')
        t_box=ax[i].text(x, y, class_name,  color='red', fontsize=10)
        t_box.set_bbox(dict(boxstyle='square, pad=0',facecolor='white', alpha=0.6, edgecolor='blue'))
        ax[i].add_patch(rect)

    ax[i].axis('off')
    ax[i].imshow(image)
    ax[i].set_xlabel('Longitude')
    ax[i].set_title(f"License: {license}")
plt.tight_layout()
plt.show()

# Extracting the 64 classes

In [ ]:
for cat in cocoTrain.cat_dict.values():
    print(cat["id"], "->", cat["name"])

1 -> person
2 -> bicycle
3 -> car
4 -> motorcycle
5 -> airplane
6 -> bus
7 -> train
8 -> truck
9 -> boat
10 -> traffic light
11 -> fire hydrant
13 -> stop sign
14 -> parking meter
15 -> bench
16 -> bird
17 -> cat
18 -> dog
19 -> horse
20 -> sheep
21 -> cow
22 -> elephant
23 -> bear
24 -> zebra
25 -> giraffe
27 -> backpack
28 -> umbrella
31 -> handbag
32 -> tie
33 -> suitcase
34 -> frisbee
35 -> skis
36 -> snowboard
37 -> sports ball
38 -> kite
39 -> baseball bat
40 -> baseball glove
41 -> skateboard
42 -> surfboard
43 -> tennis racket
44 -> bottle
46 -> wine glass
47 -> cup
48 -> fork
49 -> knife
50 -> spoon
51 -> bowl
52 -> banana
53 -> apple
54 -> sandwich
55 -> orange
56 -> broccoli
57 -> carrot
58 -> hot dog
59 -> pizza
60 -> donut
61 -> cake
62 -> chair
63 -> couch
64 -> potted plant
65 -> bed
67 -> dining table
70 -> toilet
72 -> tv
73 -> laptop
74 -> mouse
75 -> remote
76 -> keyboard
77 -> cell phone
78 -> microwave
79 -> oven
80 -> toaster
81 -> sink
82 -> refrigerator
84 -> bo

In [ ]:
# Count how many images contain each category (before filtering)
category_image_count_full = {cat_id: 0 for cat_id in cocoTrain.cat_dict.keys()}

for img_id in cocoTrain.get_imgIds():
    anns = cocoTrain.annIm_dict[img_id]
    category_ids_in_image = {ann["category_id"] for ann in anns}

    for cat_id in category_ids_in_image:
        category_image_count_full[cat_id] += 1

# Sort categories by image count (descending)
sorted_categories = sorted(category_image_count_full.items(), key=lambda x: x[1], reverse=True)

# Print the distribution
print("COCO Dataset Image Distribution (Before Filtering):\n")
print(f"Total Images: {len(cocoTrain.get_imgIds())}")
for cat_id, count in sorted_categories:
    print(f"{cocoTrain.cat_dict[cat_id]['name']}: {count} images")


COCO Dataset Image Distribution (Before Filtering):

Total Images: 118287
person: 64115 images
chair: 12774 images
car: 12251 images
dining table: 11837 images
cup: 9189 images
bottle: 8501 images
bowl: 7111 images
handbag: 6841 images
truck: 6127 images
bench: 5570 images
backpack: 5528 images
book: 5332 images
cell phone: 4803 images
sink: 4678 images
clock: 4659 images
tv: 4561 images
potted plant: 4452 images
couch: 4423 images
dog: 4385 images
knife: 4326 images
sports ball: 4262 images
traffic light: 4139 images
cat: 4114 images
umbrella: 3968 images
bus: 3952 images
tie: 3810 images
bed: 3682 images
vase: 3593 images
train: 3588 images
fork: 3555 images
spoon: 3529 images
laptop: 3524 images
motorcycle: 3502 images
surfboard: 3486 images
skateboard: 3476 images
tennis racket: 3394 images
toilet: 3353 images
bicycle: 3252 images
bird: 3237 images
pizza: 3166 images
skis: 3082 images
remote: 3076 images
boat: 3025 images
airplane: 2986 images
horse: 2941 images
cake: 2925 images
o

In [ ]:
target_classes = [
    "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "stop sign", "parking meter", "bench", "bird", "cat", "dog",
    "horse", "sheep", "cow", "elephant", "bear", "zebra", "backpack", "umbrella",
    "handbag", "tie", "skis", "sports ball", "kite", "tennis racket", "bottle",
    "wine glass", "cup", "knife", "spoon", "bowl", "banana", "apple", "orange",
    "broccoli", "hot dog", "pizza", "donut", "chair", "couch", "potted plant",
    "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "teddy bear", "hair drier"
]

# Convert COCO category names to a dictionary {name: id}
coco_categories = {cat["name"]: cat["id"] for cat in cocoTrain.cat_dict.values()}

# Find category IDs that match our selected classes
target_class_ids = [coco_categories[cls] for cls in target_classes if cls in coco_categories]

print("Selected Category IDs:", target_class_ids)


Selected Category IDs: [2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 28, 31, 32, 35, 37, 38, 43, 44, 46, 47, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 88, 89]


In [ ]:
# Count how many images contain each category
category_image_count = {cat_id: 0 for cat_id in target_class_ids}

for img_id in coco.get_imgIds():
    anns = coco.annIm_dict[img_id]
    category_ids_in_image = {ann["category_id"] for ann in anns}

    # Count each category separately
    for cat_id in category_ids_in_image:
        if cat_id in target_class_ids:
            category_image_count[cat_id] += 1

# Print category-wise image count
for cat_id, count in category_image_count.items():
    cat_name = coco.cat_dict[cat_id]["name"]
    print(f"{cat_name}: {count} images")


NameError: name 'target_class_ids' is not defined

In [ ]:
filtered_image_ids = set()

for ann in cocoTrain.annId_dict.values():
    if ann["category_id"] in target_class_ids:
        filtered_image_ids.add(ann["image_id"])

filtered_image_ids = list(filtered_image_ids)
print(f"Total Images with Target Classes: {len(filtered_image_ids)}")


Total Images with Target Classes: 104877


In [ ]:
import shutil
import os
import json
from multiprocessing import Pool

# Define dataset paths
dataset_images_dir = "./coco_train2017"
filtered_annotations_file = "./filtered_train_coco_dataset/annotations.json"

# Function to delete a single file
def delete_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

# Find all image IDs in the dataset
all_image_ids = set(cocoTrain.im_dict.keys())

# Find images that are NOT in the target filtered list
images_to_delete = all_image_ids - set(filtered_image_ids)

# Prepare deletion paths
delete_paths = [
    os.path.join(dataset_images_dir, cocoTrain.im_dict[img_id]["file_name"])
    for img_id in images_to_delete
]

# Use multiprocessing for faster deletion
with Pool(processes=8) as pool:  # Adjust based on CPU cores
    pool.map(delete_file, delete_paths)

print(f"Deleted {len(images_to_delete)} images not belonging to target classes.")

# Optimized annotations filtering
filtered_annotations = {
    "images": [cocoTrain.im_dict[img_id] for img_id in filtered_image_ids],
    "annotations": [ann for ann in cocoTrain.annId_dict.values() if ann["image_id"] in filtered_image_ids],
    "categories": [cocoTrain.cat_dict[cat_id] for cat_id in target_class_ids],
}

# Save filtered annotations
with open(filtered_annotations_file, "w") as f:
    json.dump(filtered_annotations, f, indent=4)

print(f"Filtered annotations saved to {filtered_annotations_file}")

# Remove the entire dataset folder after filtering if needed
shutil.rmtree(dataset_images_dir)
print(f"Deleted the original dataset directory: {dataset_images_dir}")


In [ ]:
# import shutil
# import os

# # Create directories for filtered images and annotations
# filtered_images_dir = "./filtered_train_coco/images"
# filtered_annotations_file = "./filtered_train_coco/annotations.json"

# os.makedirs(filtered_images_dir, exist_ok=True)

# # Copy only the relevant images
# for img_id in filtered_image_ids:
#     img_info = cocoTrain.im_dict[img_id]
#     img_filename = img_info["file_name"]
#     src_path = os.path.join(cocoTrain_images_dir, img_filename)
#     dest_path = os.path.join(filtered_images_dir, img_filename)

#     if os.path.exists(src_path):  # Ensure the image exists before copying
#         shutil.copy(src_path, dest_path)

# print(f"Copied {len(filtered_image_ids)} images to {filtered_images_dir}")

# # Save filtered annotations
# filtered_annotations = {
#     "images": [cocoTrain.im_dict[img_id] for img_id in filtered_image_ids],
#     "annotations": [ann for ann in cocoTrain.annId_dict.values() if ann["image_id"] in filtered_image_ids],
#     "categories": [cat for cat in cocoTrain.cat_dict.values() if cat["id"] in target_class_ids]
# }

# with open(filtered_annotations_file, "w") as f:
#     json.dump(filtered_annotations, f, indent=4)

# print(f"Filtered annotations saved to {filtered_annotations_file}")


In [ ]:
# import shutil
# import os
# import json
# from multiprocessing import Pool

# # Define paths
# filtered_images_dir = "./filtered_train_coco_dataset/images"
# filtered_annotations_file = "./filtered_train_coco_dataset/annotations.json"


# # Recreate directories
# os.makedirs(filtered_images_dir, exist_ok=True)

# # Function to copy a single file (for multiprocessing)
# def copy_file(src_dest):
#     src, dest = src_dest
#     if os.path.exists(src):
#         shutil.copy2(src, dest)

# # Prepare paths
# src_dest_pairs = [
#     (
#         os.path.join(cocoTrain_images_dir, cocoTrain.im_dict[img_id]["file_name"]),
#         os.path.join(filtered_images_dir, cocoTrain.im_dict[img_id]["file_name"]),
#     )
#     for img_id in filtered_image_ids
# ]

# # Use multiprocessing for faster copying
# with Pool(processes=8) as pool:  # Adjust pool size based on CPU cores
#     pool.map(copy_file, src_dest_pairs)

# print(f"Copied {len(filtered_image_ids)} images to {filtered_images_dir}")

# # Optimized annotations filtering
# filtered_annotations = {
#     "images": [cocoTrain.im_dict[img_id] for img_id in filtered_image_ids],
#     "annotations": [ann for ann in cocoTrain.annId_dict.values() if ann["image_id"] in filtered_image_ids],
#     "categories": [cocoTrain.cat_dict[cat_id] for cat_id in target_class_ids],
# }

# # Save filtered annotations
# with open(filtered_annotations_file, "w") as f:
#     json.dump(filtered_annotations, f, indent=4)

# print(f"Filtered annotations saved to {filtered_annotations_file}")
# shutil.rmtree("/content/coco_train2017")

Copied 104877 images to ./filtered_train_coco_dataset/images


In [ ]:
filtered_image_ids = set()

for ann in cocoVal.annId_dict.values():
    if ann["category_id"] in target_class_ids:
        filtered_image_ids.add(ann["image_id"])

filtered_image_ids = list(filtered_image_ids)
print(f"Total Images with Target Classes: {len(filtered_image_ids)}")


In [ ]:
import shutil
import os

# Create directories for filtered images and annotations
filtered_images_dir = "./filtered_val_coco/images"
filtered_annotations_file = "./filtered_val_coco/annotations.json"

os.makedirs(filtered_images_dir, exist_ok=True)

# Copy only the relevant images
for img_id in filtered_image_ids:
    img_info = cocoVal.im_dict[img_id]
    img_filename = img_info["file_name"]
    src_path = os.path.join(cocoVal_images_dir, img_filename)
    dest_path = os.path.join(filtered_images_dir, img_filename)

    if os.path.exists(src_path):  # Ensure the image exists before copying
        shutil.copy(src_path, dest_path)

print(f"Copied {len(filtered_image_ids)} images to {filtered_images_dir}")

# Save filtered annotations
filtered_annotations = {
    "images": [cocoVal.im_dict[img_id] for img_id in filtered_image_ids],
    "annotations": [ann for ann in cocoVal.annId_dict.values() if ann["image_id"] in filtered_image_ids],
    "categories": [cat for cat in cocoVal.cat_dict.values() if cat["id"] in target_class_ids]
}

with open(filtered_annotations_file, "w") as f:
    json.dump(filtered_annotations, f, indent=4)

print(f"Filtered annotations saved to {filtered_annotations_file}")
